In [1]:
import pandas as pd

# -----------------------------
# LOAD DATA
# -----------------------------
df = pd.read_csv("cleaned_ice_store_data.csv", keep_default_na=False)

# Replace "Closed" everywhere with NaN
df = df.replace("Closed", pd.NA)

# Fix special notes ("NA" -> "None")
df["Special Note"] = df["Special Note"].replace("NA", "None")
df["Special Note"] = df["Special Note"].fillna("None")

# -----------------------------
# DATA CLEANING
# -----------------------------
df["Date"] = pd.to_datetime(df["Date"], errors="coerce")

# Convert numeric columns
num_cols = ["Grand Sales", "High Temp", "Low Temp", "Made", "Used"]
df[num_cols] = df[num_cols].apply(pd.to_numeric, errors="coerce")

# Fill closed days with zero for production & sales
df["Grand Sales"] = df["Grand Sales"].fillna(0)
df["Made"] = df["Made"].fillna(0)
df["Used"] = df["Used"].fillna(0)

# Add closed-day flag
#df["Closed"] = (df["Grand Sales"] == 0).astype(int)

# OPTIONAL: Fill missing weather with last known value
df["High Temp"] = df["High Temp"].fillna(method="ffill")
df["Low Temp"] = df["Low Temp"].fillna(method="ffill")

# Standardize flavor column
df["Flavor"] = df["Flavor"].astype(str)

# -----------------------------
# FEATURE ENGINEERING
# -----------------------------
df["Waste"] = df["Made"] - df["Used"]
df["Usage_Efficiency"] = (df["Used"] / df["Made"]).replace([pd.NA, pd.NaT], 0)
df["Usage_Efficiency"] = df["Usage_Efficiency"].fillna(0)

df["Waste_Percentage"] = (df["Waste"] / df["Made"]).replace([pd.NA, pd.NaT], 0)
df["Waste_Percentage"] = df["Waste_Percentage"].fillna(0)


# -----------------------------
# KPI GROUPINGS
# -----------------------------
sales_by_day = df.groupby("Date")["Grand Sales"].sum().reset_index()
flavor_demand = df.groupby("Flavor")["Used"].sum().reset_index()

# -----------------------------
# CORRELATION ANALYSIS
# -----------------------------
weather_correlation = df[["High Temp", "Low Temp", "Grand Sales"]].corr()

# Encode special events
event_dummies = pd.get_dummies(df["Special Note"], prefix="Event")
df_event = pd.concat([df, event_dummies], axis=1)

# Numeric-only correlation
corr_numeric = df_event.select_dtypes(include=["number"]).corr()
specialNote_correlation = corr_numeric["Grand Sales"].sort_values(ascending=False)

# -----------------------------
# EXPORT TRANSFORMED DATASET
# -----------------------------
df.to_csv("transformed_ice_store_data.csv", index=False)

# Optional: check nulls
df.isnull().sum()

/var/folders/wj/mwd6zf295nj8ld04tk35rd240000gn/T/ipykernel_95576/1660896955.py:18: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["Date"] = pd.to_datetime(df["Date"], errors="coerce")
/var/folders/wj/mwd6zf295nj8ld04tk35rd240000gn/T/ipykernel_95576/1660896955.py:33: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df["High Temp"] = df["High Temp"].fillna(method="ffill")
/var/folders/wj/mwd6zf295nj8ld04tk35rd240000gn/T/ipykernel_95576/1660896955.py:34: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df["Low Temp"] = df["Low Temp"].fillna(method="ffill")


Day                 0
Date                0
Grand Sales         0
Special Note        0
High Temp           0
Low Temp            0
Flavor              0
Made                0
Used                0
Waste               0
Usage_Efficiency    0
Waste_Percentage    0
dtype: int64